# Data Profiling Analysis - Data Collection Data

**This notebook analyzes data from your Workbench data collection and generates a comprehensive profiling report.**

## 📊 What You'll See

After running all cells, you'll get:
- **Data Overview**: Summary statistics and data structure
- **Comprehensive Profiling Report**: Automatic analysis of all columns including:
  - Data types and missing values
  - Statistical summaries (mean, median, std, etc.)
  - Distribution visualizations
  - Correlations between variables
  - Data quality alerts

## 🚀 Quick Start

Just click **"Run All"** to analyze your data from the data collection bucket!

The profiling report works with **any data structure** - no hardcoded column names required!


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import os
from pathlib import Path
warnings.filterwarnings('ignore')

# Optional: Import google.cloud.storage (installed automatically if needed)
try:
    from google.cloud import storage
    GCS_AVAILABLE = True
except ImportError:
    GCS_AVAILABLE = False
    print("ℹ️  Installing google-cloud-storage...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "google-cloud-storage"])
    from google.cloud import storage
    GCS_AVAILABLE = True
    print("✅ google-cloud-storage installed successfully!")

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# ============================================================================
# CONFIGURATION: Data Collection Bucket and File
# ============================================================================
GCS_BUCKET = "my-gcs-experimentation-bucker-wb-steady-parsnip-7109"  # Your data collection bucket
FILE_NAME = "MUP_DPR_RY25_P04_V10_DY23_Geo.csv"   # Your data file
FILE_FORMAT = "csv"  # File format

print("="*70)
print("📊 Configuration")
print("="*70)
print(f"Bucket: {GCS_BUCKET}")
print(f"File: {FILE_NAME}")
print(f"Format: {FILE_FORMAT}")
print(f"GCS Path: gs://{GCS_BUCKET}/{FILE_NAME}")
print("="*70)


In [ ]:
## 2. Load Data from GCS Bucket


In [ ]:
def load_data_from_gcs(bucket_name, file_name, file_format="csv"):
    """Load data from GCS bucket using Google Cloud Storage client."""
    try:
        # Initialize GCS client
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(file_name)
        
        print(f"📥 Reading file from GCS: gs://{bucket_name}/{file_name}")
        
        # Download to temporary file
        temp_file = f"/tmp/{os.path.basename(file_name)}"
        blob.download_to_filename(temp_file)
        print(f"✅ File downloaded to: {temp_file}")
        
        # Read based on file format
        if file_format.lower() == "csv":
            df = pd.read_csv(temp_file)
        elif file_format.lower() == "parquet":
            df = pd.read_parquet(temp_file)
        elif file_format.lower() == "json":
            df = pd.read_json(temp_file)
        elif file_format.lower() == "excel":
            df = pd.read_excel(temp_file)
        else:
            raise ValueError(f"Unsupported file format: {file_format}")
        
        # Clean up temp file
        os.remove(temp_file)
        print(f"✅ Data loaded successfully: {len(df)} rows, {len(df.columns)} columns")
        return df
        
    except Exception as e:
        print(f"❌ Error loading from GCS: {e}")
        raise

# Load data from GCS
print("\n" + "="*70)
print("Loading data from data collection...")
print("="*70)
bucket_name = GCS_BUCKET.replace("gs://", "").strip()
df = load_data_from_gcs(bucket_name, FILE_NAME, FILE_FORMAT)

print(f"\n✅ Dataset ready: {len(df)} records")
print(f"📋 Columns: {list(df.columns)}")
print(f"\n📊 First few records:")
df.head(10)


## 3. Comprehensive Data Profiling Report


In [ ]:
# Install ydata-profiling if not available
try:
    from ydata_profiling import ProfileReport
    print("✅ ydata-profiling is available")
except ImportError:
    print("ℹ️  Installing ydata-profiling...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "ydata-profiling"])
    from ydata_profiling import ProfileReport
    print("✅ ydata-profiling installed successfully!")

# Generate comprehensive profiling report
print("\n" + "="*70)
print("📊 Generating Comprehensive Data Profiling Report...")
print("="*70)
print("This may take a few moments depending on your data size...")

# Create profile report
# Set minimal=True for very large datasets (>100k rows) for faster processing
profile = ProfileReport(
    df,
    title="Data Profiling Report",
    minimal=False,  # Set to True for very large datasets
    progress_bar=True,
    html={'style': {'full_width': True}}
)

# Display the report
profile.to_notebook_iframe()

# Optionally save the report as HTML file
# profile.to_file("data_profile_report.html")
# print("\n✅ Report saved as 'data_profile_report.html'")

print("\n" + "="*70)
print("✅ Profiling Report Complete!")
print("="*70)


In [ ]:
 
            startangle=90, textprops={'fontsize': 9})
axes[1].set_title('Lab Type Distribution (Percentage)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
print("=== Lab Value Distribution Report ===")
print(f"\nBasic Statistics:")
print(df['Lab Value'].describe())
print(f"\nSkewness: {df['Lab Value'].skew():.3f}")
print(f"Kurtosis: {df['Lab Value'].kurtosis():.3f}")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Histogram
axes[0, 0].hist(df['Lab Value'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].set_xlabel('Lab Value', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Lab Value Distribution (Histogram)', fontsize=14, fontweight='bold')
axes[0, 0].axvline(df['Lab Value'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["Lab Value"].mean():.2f}')
axes[0, 0].axvline(df['Lab Value'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df["Lab Value"].median():.2f}')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Box plot
axes[0, 1].boxplot(df['Lab Value'], vert=True, patch_artist=True, 
                   boxprops=dict(facecolor='lightblue', alpha=0.7))
axes[0, 1].set_ylabel('Lab Value', fontsize=12)
axes[0, 1].set_title('Lab Value Distribution (Box Plot)', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Density plot
df['Lab Value'].plot.density(ax=axes[1, 0], color='purple', linewidth=2)
axes[1, 0].set_xlabel('Lab Value', fontsize=12)
axes[1, 0].set_ylabel('Density', fontsize=12)
axes[1, 0].set_title('Lab Value Distribution (Density Plot)', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Lab values by lab type
df.boxplot(column='Lab Value', by='Lab Type', ax=axes[1, 1], rot=45)
axes[1, 1].set_xlabel('Lab Type', fontsize=10)
axes[1, 1].set_ylabel('Lab Value', fontsize=12)
axes[1, 1].set_title('Lab Value Distribution by Lab Type', fontsize=14, fontweight='bold')
plt.setp(axes[1, 1].xaxis.get_majorticklabels(), rotation=45, ha='right')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


In [ ]:
print("="*70)
print(" " * 15 + "📊 COMPREHENSIVE SUMMARY REPORT 📊")
print("="*70)
print(f"\n{'='*70}")
print(f"📋 DATASET OVERVIEW")
print(f"{'='*70}")
print(f"  Total Records: {len(df):,}")
print(f"  Date Range: {df['Lab Date'].min().date()} to {df['Lab Date'].max().date()}")
print(f"  Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n{'='*70}")
print(f"👥 PATIENT ID ANALYSIS")
print(f"{'='*70}")
print(f"  Total Unique Patients: {df['Patient ID'].nunique():,}")
print(f"  Average Tests per Patient: {df.groupby('Patient ID').size().mean():.2f}")
print(f"  Median Tests per Patient: {df.groupby('Patient ID').size().median():.2f}")
print(f"  Patients with Most Tests: {df.groupby('Patient ID').size().max()} tests")

print(f"\n{'='*70}")
print(f"🧪 LAB TYPE ANALYSIS")
print(f"{'='*70}")
print(f"  Total Unique Lab Types: {df['Lab Type'].nunique()}")
most_common = df['Lab Type'].mode()[0]
most_common_count = df['Lab Type'].value_counts().max()
most_common_pct = (most_common_count / len(df)) * 100
print(f"  Most Common Lab Type: {most_common} ({most_common_count} tests, {most_common_pct:.1f}%)")
print(f"  Lab Type Distribution:")
for lab_type, count in df['Lab Type'].value_counts().head(5).items():
    pct = (count / len(df)) * 100
    print(f"    - {lab_type}: {count} ({pct:.1f}%)")

print(f"\n{'='*70}")
print(f"📈 LAB VALUE STATISTICS")
print(f"{'='*70}")
print(f"  Mean: {df['Lab Value'].mean():.2f}")
print(f"  Median: {df['Lab Value'].median():.2f}")
print(f"  Standard Deviation: {df['Lab Value'].std():.2f}")
print(f"  Minimum: {df['Lab Value'].min():.2f}")
print(f"  Maximum: {df['Lab Value'].max():.2f}")
print(f"  Range: {df['Lab Value'].max() - df['Lab Value'].min():.2f}")
print(f"  Skewness: {df['Lab Value'].skew():.3f}")

print(f"\n{'='*70}")
print(f"📅 TEMPORAL ANALYSIS")
print(f"{'='*70}")
date_range_days = (df['Lab Date'].max() - df['Lab Date'].min()).days
print(f"  Date Range: {date_range_days} days")
print(f"  Average Tests per Day: {len(df) / (date_range_days + 1):.2f}")
print(f"  Total Tests: {len(df):,}")

# Find busiest day
daily_counts = df.groupby(df['Lab Date'].dt.date).size()
busiest_day = daily_counts.idxmax()
busiest_count = daily_counts.max()
print(f"  Busiest Day: {busiest_day} ({busiest_count} tests)")

print(f"\n{'='*70}")
print("✅ Analysis Complete! All distribution reports and visualizations are shown above.")
print(f"{'='*70}")
